## Task 2 :

### pre-analysis:

In [83]:
import pandas as pd

df = pd.read_csv("task2.csv")

df['Company_Size'] = df['Company_Size'].replace('S', 'Small')
columns_to_check = ['Work_Year', 'Experience_Level', 'Employment_Type', 'Job_Title', 'Salary', 'Salary_Currency', 'Remote_Ratio', 'Company_Size']

for column in columns_to_check:
    missing_values = df[column].isnull().sum()
    unique_values = df[column].unique()
    print(f"Column: {column}")
    print(f"Number of missing values: {missing_values}")
    print(f"Unique values: {unique_values}")
    print("\n")


Column: Work_Year
Number of missing values: 0
Unique values: [2024 2023 2022 2020 2021]


Column: Experience_Level
Number of missing values: 0
Unique values: ['Senior-Level' 'Mid-Level' 'Entry-Level' 'Executive-Level']


Column: Employment_Type
Number of missing values: 0
Unique values: ['Full-Time' 'Contract' 'Part-Time' 'Freelance']


Column: Job_Title
Number of missing values: 0
Unique values: ['AI Engineer' 'Machine Learning Engineer'
 'Business Intelligence Developer' 'Data Engineer' 'Data Scientist'
 'Cloud Database Engineer' 'Research Engineer' 'Data Analyst'
 'Machine Learning Scientist' 'Applied Scientist' 'Data Science Manager'
 'Research Scientist' 'Prompt Engineer' 'Data Science'
 'Data Science Consultant' 'Data Management Analyst' 'Research Analyst'
 'Data Operations Analyst' 'Data Management Consultant'
 'Business Intelligence Analyst' 'Analytics Engineer'
 'Data Quality Analyst' 'Data Architect' 'Data Manager' 'ML Engineer'
 'Robotics Software Engineer' 'Machine Learning

In [84]:
duplicate_rows = df[df.duplicated()]
if not duplicate_rows.empty:
    print("Number of duplicate rows :",len(duplicate_rows))
else:
    print("No duplicate found!")

Number of duplicate rows : 5565


In [85]:
df = df.drop_duplicates()

In [86]:
currency_counts = df['Salary_Currency'].value_counts()
print("Currency Counts befroe removing:")
print(currency_counts,"\n")

currencies_to_remove = currency_counts[currency_counts < 10].index
df = df[~df['Salary_Currency'].isin(currencies_to_remove)]

currency_counts = df['Salary_Currency'].value_counts()
print("Currency Counts after removing:")
print(currency_counts)

Currency Counts befroe removing:
Salary_Currency
USD    7499
GBP     410
EUR     339
INR      52
CAD      48
AUD      12
PLN       7
CHF       6
SGD       6
JPY       4
BRL       4
DKK       3
HUF       3
TRY       3
NOK       2
THB       2
ZAR       1
HKD       1
ILS       1
PHP       1
NZD       1
MXN       1
CLP       1
Name: count, dtype: int64 

Currency Counts after removing:
Salary_Currency
USD    7499
GBP     410
EUR     339
INR      52
CAD      48
AUD      12
Name: count, dtype: int64


In [87]:
df.to_csv("task2_cleaned.csv")

### Changing currencies to USD 

In [104]:
import pandas as pd
from forex_python.converter import CurrencyRates

df = pd.read_csv("task2_cleaned.csv")

currencies = df['Salary_Currency'].unique()
print(currencies)

['USD' 'EUR' 'GBP' 'CAD' 'AUD' 'INR']


In [105]:
exchange_rates = {}

cr = CurrencyRates()
for currency in currencies:
    if currency != 'USD':
        exchange_rates[currency] = cr.get_rate(currency, 'USD')

In [106]:
print(exchange_rates)

{'EUR': 1.0768, 'GBP': 1.2578704514923194, 'CAD': 0.7416999586719933, 'AUD': 0.6519343706484229, 'INR': 0.012047235725513668}


In [107]:
def convert_to_usd(amount, currency):
    if currency != 'USD':
        return round(amount * exchange_rates[currency])
    else:
        return amount

df['Salary'] = df.apply(lambda row: int(convert_to_usd(row['Salary'], row['Salary_Currency'])), axis=1)

df['Salary_Currency'] = 'USD'

In [108]:
df.to_csv("task2_USD.csv")